In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
df = pd.read_excel(
    "/Users/jamesliounis/Documents/Harvard SM DS/NCE512/NCE_512_Problem_Sets/NCE512_PS2/Pset2_data.xlsx"
)

In [3]:
df.head()

,Wait,x_ArrivalDTTM,x_ScheduledDTTM,x_BeginDTTM,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,2025-08-02 16:47:44,2025-08-02 17:45:00,2025-08-02 17:45:03,372,62,6,249,6,5,...,4,2,0,0,0,0,3,1,1,0
1,-3,2025-08-02 16:38:46,2025-08-02 17:15:00,2025-08-02 17:12:14,335,67,3,195,5,4,...,4,1,2,0,32,0,2,1,2,0
2,-25,2025-08-02 16:29:25,2025-08-02 17:30:00,2025-08-02 17:05:15,274,68,4,150,4,5,...,4,1,1,1,13,16,1,1,3,0
3,36,2025-08-02 16:16:24,2025-08-02 16:45:00,2025-08-02 17:21:55,369,73,2,179,5,6,...,3,1,2,1,45,16,3,1,1,0
4,-21,2025-08-02 16:04:36,2025-08-02 17:15:00,2025-08-02 16:54:49,438,73,5,192,6,5,...,3,0,2,1,32,15,4,0,0,1


In [4]:
# Dropping columns as per instructions
df.drop(['x_ArrivalDTTM', 'x_ScheduledDTTM',	'x_BeginDTTM'], axis=1, inplace=True)

In [5]:
df.head()

,Wait,SumHowEarlyWaiting,AvgHowEarlyWaiting,LineCount0Strict,SumWaits,LineCount0,LineCount1,LineCount2,LineCount3,LineCount4,...,NumAddOnsToday,NumAddOnsLastW2,NumScheduledNextSlot,NumScheduledNextW2,SumTimeToCompleteNextSlot,SumTimeToCompleteNextW2,WithContrastCountWaiting,WithAndWithoutContrastCountWaiting,WithContrastCountInProgress,WithAndWithoutContrastCountInProgress
0,0,372,62,6,249,6,5,6,6,4,...,4,2,0,0,0,0,3,1,1,0
1,-3,335,67,3,195,5,4,6,6,5,...,4,1,2,0,32,0,2,1,2,0
2,-25,274,68,4,150,4,5,6,4,6,...,4,1,1,1,13,16,1,1,3,0
3,36,369,73,2,179,5,6,4,5,5,...,3,1,2,1,45,16,3,1,1,0
4,-21,438,73,5,192,6,5,5,6,5,...,3,0,2,1,32,15,4,0,0,1


# Linear Regression

**Question 1**

Using sklearn snippet above, build a linear regression model to predict Wait from all other columns (excluding the “x_” columns, as I mentioned). Once the model is built, use it to compute the predicted wait value Ypred, and residual r (the difference between the true and predicted wait).

 

Let’s define model error  as the average of its absolute residual, as shown above: abs(r).mean()

 

Find this error value e, and write it down.

 

What was the model error value ?

In [6]:
X = np.array(df.drop('Wait', axis=1))
y = df['Wait']

model = LinearRegression()
model.fit(X, y)
Ypred = model.predict(X)
r = y-Ypred
e = abs(r).mean()
e

23.16628893634662

In [7]:
median =  abs(r).median()
median

19.126063446192802

**Question 2**

Using the code below, understand and run Python built-in feature selection algorithm: RFE (Recursive Feature Elimination. In this code, “model” is the same regression model as you declared and used in the previous code.

Write down its output. What was the best 3-feature model error value?

In [11]:
# Run Python feature selection

if True: # just in case I want to disable this part
    print('\n>Python feature selection:')
    from sklearn.feature_selection import RFE
    from itertools import compress
    for nFeatures in range(1,4):
        rfe = RFE(model, n_features_to_select=nFeatures)
        X_rfe = rfe.fit_transform(X,y) #transforming data using RFE   
        #Fitting the data to model
        model.fit(X_rfe,y)
        #print(rfe.support_)
        #print(rfe.ranking_)
        cols = list(compress( pd.DataFrame(X).columns, rfe.support_))
        model.fit(pd.DataFrame(X)[cols],y)
        e = abs(y-model.predict(pd.DataFrame(X)[cols])).mean()
        print(e, cols)



>Python feature selection:
32.50541172120526 [25]
31.224937706982562 [25, 49]
31.25752894868893 [25, 37, 49]


# Stepwise regression

**Question 3**

Can we always rely on Python built-in feature selectors? Let’s implement our own best model selection as a fast-forward “greedy” algorithm[1]. It should work like this:

Find the best one-feature model (try all one-feature models, and select the one with the lowest error e). This is our best feature F1.
Using F1 from the first step, try adding one more feature to it (from all features you have left), to find the best 2-feature model (F1, F2)
Similarly, keep adding more features: F3, F4, F5 – to the features from the previous step
What is the e=abs(r).mean() value for the best 3-feature model?

In [16]:
def stepwise_regression(X, y, error_metric='mae'):
    
    n_features = X.shape[1]
    remaining_features = set(range(n_features))
    selected_features = []
    
    best_error = np.inf
    
    while remaining_features:
        for feature in remaining_features:
            features_to_try = selected_features + [feature]
            X_train = X[:, features_to_try]
            model = LinearRegression().fit(X_train, y)
            y_pred = model.predict(X_train)
            error = np.mean((y - y_pred) ** 2) if error_metric == 'mse' else np.mean(np.abs(y - y_pred))
            
            if error < best_error:
                best_error = error
                best_feature = feature
        try:
            remaining_features.remove(best_feature)
            selected_features.append(best_feature)
            if len(selected_features) == 3:
                print (f'Best 3 feature model: {selected_features}, Error: {best_error:.3f}')
            elif len(selected_features) <=15:
                print((f'Less than 15 feature model: {selected_features}, Error: {best_error:.3f}'))
            # print(f'Selected features: {selected_features}, Error: {best_error:.3f}')
        except KeyError:
            print('We have reached the maximum number of selected features.')
            break
        
    return selected_features




In [17]:
stepwise_regression(X, y)

Less than 15 feature model: [49], Error: 31.227
Less than 15 feature model: [49, 2], Error: 30.138
Best 3 feature model: [49, 2, 4], Error: 28.696
Less than 15 feature model: [49, 2, 4, 51], Error: 28.246
Less than 15 feature model: [49, 2, 4, 51, 19], Error: 28.147
Less than 15 feature model: [49, 2, 4, 51, 19, 62], Error: 24.990
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78], Error: 24.659
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15], Error: 24.329
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15, 58], Error: 24.072
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15, 58, 40], Error: 23.847
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15, 58, 40, 72], Error: 23.693
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15, 58, 40, 72, 59], Error: 23.628
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15, 58, 40, 72, 59, 52], Error: 23.579
Less than 15 feature model: [49, 2, 4, 51, 19, 62, 78, 15, 58, 40, 72, 59, 52, 77], Error: 23

[49,
 2,
 4,
 51,
 19,
 62,
 78,
 15,
 58,
 40,
 72,
 59,
 52,
 77,
 42,
 68,
 45,
 50,
 12,
 63,
 82,
 81,
 30,
 39,
 41,
 61,
 64,
 25,
 16,
 48,
 53,
 56,
 29,
 11,
 55,
 27,
 38,
 37,
 13,
 7,
 74,
 28,
 10,
 43,
 17,
 44,
 33,
 75,
 26,
 21,
 46,
 83,
 31,
 18,
 22,
 79,
 80,
 47,
 14,
 73,
 71,
 70,
 3,
 1,
 0,
 57,
 84,
 5,
 35,
 54,
 9,
 23,
 24,
 20]

This function performs stepwise regression on a given input data X and target y. Stepwise regression is a technique used to build a model by iteratively adding or removing features until a desired level of model performance is achieved.

The function takes three parameters: X, y, and error_metric. X is a numpy array that represents the input features of the model, while y is a numpy array that represents the target variable. error_metric is a string that indicates the error metric to be used for the regression. By default, it is set to 'mae' (mean absolute error), but it can also be set to 'mse' (mean squared error).

The function starts by initializing some variables, such as the number of features in the input data and the set of remaining features that have not been selected yet. It then enters a while loop that will continue until all features have been selected.

Inside the while loop, the function iterates over the remaining features and adds each one to the selected features to create a new model. It then fits a linear regression model to the new data and computes the error using the specified error metric. If the error is smaller than the current best error, the new feature is selected as the best feature.

Once the best feature has been selected, it is removed from the set of remaining features and added to the list of selected features. If the number of selected features is 3, the function prints the feature list and the associated error. If the number of selected features is less than or equal to 15, the function also prints the feature list and the associated error. Finally, the function returns the list of selected features.

Overall, this function performs stepwise regression by iteratively selecting the best feature to add to the model until a desired level of performance is achieved. It can be useful for feature selection in cases where the number of input features is large and the desired level of model performance is not immediately clear.